In [1]:
# 安裝必要套件
!pip install llama-cpp-python chromadb faiss-cpu PyMuPDF --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103

In [2]:
# 下載 TinyLLaMA 1.1B Q4_K_M 量化模型 (~700MB)
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf
# 下載 Qwen1.5-0.5B-Chat Q4_K_M 量化模型（中文支援）
#!wget -q https://huggingface.co/Qwen/Qwen1.5-0.5B-Chat-GGUF/resolve/main/qwen1_5-0_5b-chat-q4_k_m.gguf -O qwen-0.5b-chat.gguf


In [3]:
from llama_cpp import Llama

# 載入模型
llm = Llama(model_path="tinyllama.gguf", n_ctx=2048, n_threads=4)
#llm = Llama(model_path="qwen-0.5b-chat.gguf", n_ctx=2048, n_threads=4)
print("模型已載入完成")

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from tinyllama.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count

模型已載入完成


Using gguf chat template: {% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<|user|>
' + message['content'] + eos_token }}
{% elif message['role'] == 'system' %}
{{ '<|system|>
' + message['content'] + eos_token }}
{% elif message['role'] == 'assistant' %}
{{ '<|assistant|>
'  + message['content'] + eos_token }}
{% endif %}
{% if loop.last and add_generation_prompt %}
{{ '<|assistant|>' }}
{% endif %}
{% endfor %}
Using chat eos_token: </s>
Using chat bos_token: <s>


In [4]:
from chromadb import Client
import chromadb.utils.embedding_functions as ef

embedding_func = ef.DefaultEmbeddingFunction()
chroma_client = Client()
collection = chroma_client.create_collection(name="docs", embedding_function=embedding_func)
print("已建立 ChromaDB 檢索資料庫")


已建立 ChromaDB 檢索資料庫


In [5]:
def rag_query(query):
    results = collection.query(query_texts=[query], n_results=3)
    context = " ".join(results["documents"][0])
    prompt = f"""使用以下資料回答問題：
{context}

問題：{query}
回答：
"""
    output = llm(prompt, max_tokens=256, stop=["</s>"])
    return output["choices"][0]["text"]


In [6]:
print(rag_query("國家科學及技術委員會補助國外學者的經費規定"))

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 101MiB/s]
llama_perf_context_print:        load time =    4144.27 ms
llama_perf_context_print: prompt eval time =    4144.00 ms /    62 tokens (   66.84 ms per token,    14.96 tokens per second)
llama_perf_context_print:        eval time =   23361.75 ms /   255 runs   (   91.61 ms per token,    10.92 tokens per second)
llama_perf_context_print:       total time =   27685.79 ms /   317 tokens


1. 國家科學及技術委員會經費規定
    - 國家科學及技術委員會會作出經費規定，以認證科學及技術人員的資格、經驗和水平。
    - 經費規定應規劃足以監督與認證科學及技術人員的狀況，以保證科學及技術研究成果能夠與科學研究所於其中的其他類型的科學和技術研究產品所有權持有者具有依法之權益。

2. 國家科學及技術委員會規定對各級儀度
    - 國家


In [7]:
from google.colab import files
import fitz  # PyMuPDF

uploaded = files.upload()
for filename in uploaded.keys():
    if filename.lower().endswith(".pdf"):
        doc = fitz.open(filename)
        text = ""
        for page in doc:
            text += page.get_text()
    elif filename.lower().endswith(".txt"):
        with open(filename, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        print(f"不支援的檔案格式: {filename}")
        continue

    # 分段加入向量資料庫
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    ids = [f"{filename}_{i}" for i in range(len(chunks))]
    collection.add(documents=chunks, ids=ids)
    print(f"已處理並加入知識庫: {filename}")

print("文件已全部加入知識庫")


Saving 國家科學及技術委員會補助國外學者專家來臺從事科技合作研究活動支付費用最高標準表1110830 (3).pdf to 國家科學及技術委員會補助國外學者專家來臺從事科技合作研究活動支付費用最高標準表1110830 (3).pdf
已處理並加入知識庫: 國家科學及技術委員會補助國外學者專家來臺從事科技合作研究活動支付費用最高標準表1110830 (3).pdf
文件已全部加入知識庫


In [9]:

print(rag_query("國家科學及技術委員會補助國外學者的經費規定"))


Llama.generate: 1479 prefix-match hit, remaining 24 prompt tokens to eval
llama_perf_context_print:        load time =    4144.27 ms
llama_perf_context_print: prompt eval time =    1241.59 ms /    24 tokens (   51.73 ms per token,    19.33 tokens per second)
llama_perf_context_print:        eval time =   30298.79 ms /   257 runs   (  117.89 ms per token,     8.48 tokens per second)
llama_perf_context_print:       total time =   31719.80 ms /   281 tokens


上未滿一年者，依本表所列項目，並實際需要於所在合作機構專案報請至
多趟為限，得依實際需要於所在合作機構專案報請至多二趟來回
機票款。 
六、 本表機票補助僅為國外學者專家本人之來回機票款，單項計畫以一趟為
限；惟核定在臺灣日數達四個月以上者，得依實際需要於所在合作
機構專案報請至多二趟來
